# Data import

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c635021591574cbbbe02d56a07fdd8b0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PK2r7dODqrUrMdRBC71OHeuQb9NEHLvx_tJ5uiR212cg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c635021591574cbbbe02d56a07fdd8b0.get_object(Bucket='accident-donotdelete-pr-ejjo8tlwei04cp',Key='Data-Collisions.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


# Preprocessing

In [11]:
df_data_1['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

As you can see, the number of rows in class 1 is almost three times bigger than the number of rows in class 2. It is possible to solve the issue by downsampling the class 1.

In [12]:
from sklearn.utils import resample

In [14]:
df_data_1_maj=df_data_1[df_data_1.SEVERITYCODE==1]
df_data_1_min=df_data_1[df_data_1.SEVERITYCODE==2]
df_data_1_maj_dsample=resample(df_data_1_maj,replace=False,n_samples=58188,random_state=123)
balanced_df=pd.concat([df_data_1_maj_dsample,df_data_1_min])
balanced_df.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

# Methodology

In [15]:
df_data_1["WEATHER"].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [17]:
df_data_1["ROADCOND"].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [18]:
df_data_1["LIGHTCOND"].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [21]:
##split to train and test

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_data_1["LIGHTCOND"], df_data_1["SEVERITYCODE"], test_size=0.33, random_state=42)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k=17
knn=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
knn_y_pred=knn.predict(X_test)
knn_y_pred[0:5]

# KNN evaluation

In [ ]:
jaccard_score(y_test,knn_y_pred)

In [ ]:
f1_score(y_test,knn_y_pred,average='macro')

# DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(criterian='entropy',max_depth=7)
dt.fit(X_train,y_train)

In [ ]:
dt_y_pred=dt.predict(X_test)

# DecisionTreeEvaluation

In [ ]:
jaccard_score(y_test,dt_y_pred)

In [ ]:
f1_score(y_test,dt_y_pred,average='macro')

# linearregression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
lr=LogisticRegression(C=6,solver='liblinear').fit(X_train,y_train)

In [ ]:
lr_y_pred=lr.predict(X_test)
lr_y_prob=lr.predict_proba(X_test)
log_loss(Y_test,lr_y_prob)

# linear regression evaluation

In [ ]:
jaccard_score(y_test,lr_y_pred)

In [ ]:
f1_score(y_test,lr_y_pred,average='macro')

# result and evaluation

basing on what we have seen so far , knn model is the best suited model

# concluision

Based on the dataset provided for this capstone from weather, road, and light conditions pointing to certain classes, we can conclude that particular conditions have a somewhat impact on whether or not travel could result in property damage (class 1) or injury (class 2).